# Modular Division

##### This file outlines the essentials of Division and Multaplicative Inverses, when it comes to Modular arithmetic.
Written by Brandon Allen

# Template

Quick access to 3 pieces of code:

* Modular inverse (using Extended Euclidean Algorithm)
* Modular inverse (Using Euler's Theorem)
* Modular Division

(O(1) time and space complexity (given integers are bounded to a constant))

In [2]:
import math
# Modular Inverse
# Using Extended Euclidean Algorithm

# O(log(min(a, mod)))
def xgcd(a, mod):
    x, y = 0, 1
    lx, ly = 1, 0
    while a:
        mod, q, a = a, mod//a, mod%a
        x, lx = lx, x - q*lx
        y, ly = ly, y - q*ly
    return mod, x, y

# Given two intevers a and mod, find the modular inverse of a under modulo mod.
# O(log(a))
def modinv(a, mod):
    g, x, _ = xgcd(a, mod)
    if g != 1:
        raise ValueError(f'{a} has no inverse modulo {mod} because they are not coprime')
    return x%mod

In [49]:
# Modular Inverse
# Using Euler's Theorem
import math
from functools import cache
# Totient Function
#Find the number of coprime numbers to n less than n
#Can substitute logic for a pre-cached sieve if this is being called many times
#O(sqrt(n))
@cache
def totient(n):
    ret = n
    fac = 2
    while n > 1 and fac*fac <= n:
        if not n%fac:
            ret *= fac-1
            ret//=fac
            n//=fac
            while not n%fac:
                n//=fac
        fac+=1
    if n > 1:
        ret *= n-1
        ret//=n
    return ret

#Given two integers a and mod, find the modular inverse of a under modulo mod.
#O(sqrt(n))
def modinv(a, mod):
    if math.gcd(a, mod) != 1:
        raise ValueError(f'{a} has no inverse modulo {mod} because they are not coprime')
    return pow(a, totient(mod)-1, mod)

In [32]:
#Euler's Theorem Shorthand
# ** Only works if mod is prime **
#O(log(mod))
def modinv(a, mod):
    return pow(a, mod - 2, mod)

In [47]:
# Modular Division
import math

def xgcd(a, mod):
    x, y = 0, 1
    lx, ly = 1, 0
    while a:
        mod, q, a = a, mod//a, mod%a
        x, lx = lx, x - q*lx
        y, ly = ly, y - q*ly
    return mod, x, y


def modinv(a, mod):
    g, x, _ = xgcd(a, mod)
    if g != 1:
        raise ValueError(f'{a} has no inverse modulo {mod} because they are not coprime')
    return x%mod

# Given three integers a, b, and mod, find the modular division of a/b modulo mod.
#O(log(mod))
def moddiv(a, b, mod):
    return a*modinv(b, mod)%mod

#Can also call pow(a, -1, mod) in Python >= v3.8

## FAQ

Below are nuances and clarifications on Modular Division. If you're looking for a basic understanding, skip to the Guide section.

Q: What is a modular multiplicative inverse?

* The inverse of a number A, with a given modulus M, is a number B, such that (A * B)%m == 1. This is only possible if A and M are coprime.

Q: What's the realistic use-case for a modular multiplicative inverse?

* Most of the time modular multiplicative inverses are used to divide two numbers, that have been expressed as the remainder of some coprime modulus. If A and B are expressed as remainders of some M, regular arithmetic division cannot calculate (A/B)%M, given A%M and B%M. Rather than attempting regular arithmetic division to find C, one can simply find the multiplicative inverse of B mod(M), and multiply A by this multiplicative inverse. This shows up in number theory, cryptography, and is common in competitive programming. Most importantly it is fundamental to understanding the internals workings of hashing algorithms.

Q: Why are only some elements invertible, given a modulus?

* An invertible element A, by definition, has a multiplicative inverse B for A given M where (A * B)%M == 1. If A isn't coprime with M, then there is no B for which (A * B)%M == 1. For example, if A == 4, and M == 12, any product with A mod(M) of A will be in the set {4, 8, 0}. The number of invertible elements mod(M) is the amount of numbers below M, that are coprime with M, or ϕ(M).

Q: Why does th Euler's Theorem approach use Euler's Totient Function?

* Euler's Totient function ϕ(M) returns the number of integers up to M that are coprime to M. Euler's Theorem states for any A and M that are coprime, (A<sup>ϕ(M)</sup>)%M == 1. So to find the modular multiplicative inverse of A, we can find A<sup>ϕ(M)-1</sup>, which, when multiplied with A, will be 1. For any A that is coprime with M, the length of the cycle formed by powers of A will be a factor of ϕ(M). When M is prime this ring's length is exactly ϕ(M), or M-1, but with non-prime moduli, there could be multiple rings with differing lengths. Regardless, in order to obtain the last element in the ring(A, M) before 1, one can simply find A<sup>ϕ(M)-1</sup>, without ever deriving the length of the given ring formed by A mod M. The formula for ϕ(M) used above is Euler's product formula.

Q: Why does the approach using Euler's Theorem have a conditional shorthand for prime moduli?

* As described below, a number and its modular multiplicative inverse cycle through finite rings in either direction, given a modulus. The rings are isomorphic to the multiplicative relationship between integers and their multiples in regular arithmetic. For example, the powers of two are 2, 4, 8, 16, 32, 64, 128, etc. Taken modulo 3, this maps to 2:2, 4:1, 8:2, 16:1, ... 1, 2, 1, 2, 1, 2, 1 etc. For non-prime moduli, the amount and lengths of these rings are variable, and depend on the primitive roots of a modulus. But for prime moduli, there is only 1 ring, and its cycle is length M-1. Above, when ϕ(M)-1 is used to find A<sup>-1</sup>, when ϕ(M) == M-1 (e.g. when M is prime), we can simplify ϕ(M)-1 to m-2, and simply take A<sup>M-2</sup> (%M) to find its modular multiplicative inverse, A<sup>-1</sup>.

## Guide

Below is a detailed explanation of exactly what a modular inverse is, how modular division works, and a refresher on other modular operations.

This article so far talks about the difficult part of modular arithmetic; the part that involves calculating and using modular inverses. Everything else is fairly straightforward, because the homomorphisms between modular arithmetic and regular arithmetic are straightforward:

For addition, subtraction and multiplication:

* (A+B)%M = (A%M + B%M)%M

* (A-B)%M = (A%M - B%M)%M

* (A*B)%M = (A%M * B%M)%M

For example, say A = 45, B = 76, M = 7:

* A+B = (45+76)%7 = 121%7 = 2.

    ((A%7)+(A%7))%7 = (45%7 + 76%7)%7 = (3+6)%7 = 9%7 = 2.

* (A-B)%7 = (45-76)%7 = -31%7 = 4.

    ((A%7)-(B%7))%7 = (45%7 - 76%7)%7 = (3-6)%7 = -3%7 = 4.

* A * B = (45 * 76)%7 = (3420)%7 = 4.

    ((A%7) * (B%7))%7 = (45%7 * 76%7)%7 = (3 * 6)%7 = 18%7 = 4.

This just leaves division as tricky, both functionally and conceptually.

For example, say A = 540, B = 45, M = 7:

We know that A/B = 540/45 = 12, and 12%7 = 5.

But what if we already took 540%7 = 1, and 45%7 = 3?

Now we're trying to find 1/3, and we're expecting 5. What does the expression '1/3 = 5' even mean in this context? Is it even possible to reliably derive 5 from 1/3?

Yes, if the modulus is 7, we can assert that the result will always be 5 when A and B are 1 and 3, regardless of the original numbers before the modulo operation. Another quick example to illustrate this: A = 57, B = 3, M = 7: (57/3)%7 = 5.

Really what the expression (1/3) (M 7) means here is, for all integers A/B where A%7 == 1 and B%7 == 3, (A/B)%7 == 5.

This is where the *modular muliplicative inverse* comes in. When a given remainder is multiplied by its inverse, the result is 1. For example, when M is 7, the modular multiplicative inverse of 2 is 4, because 2 * 4 = 8%7 = 1. No matter whether it's 2 * 4, 2 * 11, 2 * 18, 9 * 18, etc. the result will always be 1 when taken modulo 7.

This can be likened to a multiplicative inverse to a matrix; in this separate arithmetic system (in this case modular arithmetic; in the case of matrices, matrix arithmetic), the inverse of a number is the number that, when multiplied by the original number, results in the identity element of the system (in this case, 1; in the case of matrices, the identity matrix). Just like the multiplicative inverse of a matrix, this is also called the reciprocal.

Similar to how multiplying by the reciprocal of a number is the same as dividing by that number (in regular arithmetic), multiplying A by the modular inverse of B is the same as finding (X/X)%m, where X%M == A and Y%M == b, and X is divisibly by Y.

An important part of modular arithmetic is the cyclical nature of its operations. For the hours on a clock (i.e. M = 12), every time you add A to a number, it will wrap around to be somewhere in the range from 0 to 11. There is another cyclical pattern that appears. If you start with any number, and add increments of 3 hours, it will always wrap around to the same number every 4 increments (which corresponds to 12/3= 4). If you add increments of 4 hours, you will reach the same number every 3 increments. If you add increments of 7 hours (which is relatively prime to 12), it will take 12 increments to reach the same number; 7, 2, 9, 4, 11, 6, 1, 8, 3, 10, 5, 0, 7, 2 ... with a length of M/gcd(A, M).

This cyclical quality is also true with multiplication. With all coprime elements below 12, when you multiply a number by A, it will repeat the same pattern of numbers. It's worth noting that the length of the ring formed will be a different function of A and M.

If we start with A=1, and repeatedly multiply it by 7 and take mod 12, we get 1, 7, 1, 7 ...

This doesn't cycle through every number below 12, because 12 isn't prime. If we took A = 1, B = 7 and M = 11, we would get 1, 7, 5, 2, 3, 10, 4, 6, 9, 8, 1, 7 ... which cycles through 10 numbers. For 8, we get 1, 8, 9, 6, 4, 10, 3, 2, 5, 7, 1, 8, .... which is the same cycle backwards. We can double check that 8 is the modular inverse of 7 by comfirming that (7 * 8)%56 = 1.

If we think about what B and its modular inverse for 11 (7 and 8) are doing, they're traversing through elements in opposite directions across the same cycle. This is a way of intuiting what it means for 7 and 8 to be modular multplicative inverses of each other. Multiplying any X by 7 (mod 11) is the same as dividing any Y that's divisible by 8 (mod 11) by 8, if X%M == Y%M.

For example, 1234567%11 = 4, and 936720%11 = 4.

1234567 * 7 = 8641969, and 8641969%11 = 6.

936720 / 8 = 117090, and 117090%11 = 6.

In a simplistic case where many numbers have been hashed with chain multiplication under a modulus, one could extract numbers from the hash, regardless of size, using the modular multiplicative inverse of remainders. In an array of 10<sup>10</sup> numbers each bounded to 10<sup>10</sup>, this allows the hash to be bounded to log<sub>2</sub>(M) bits, rather than log<sub>2</sub>((10<sup>10</sup>)<sup>10<sup>10</sup></sup>) bits, which is the difference between less than 100 bits (given a prime with 30 digits), and 12 gigabytes.

That is exactly why, and how, one would use the modular multiplicative inverse to perform modular division.

References:

1. Stack Overflow. (2019, May 28). How to calculate inverse modular exponentiation in C. Retrieved October 16, 2023, from https://stackoverflow.com/questions/56335032/how-to-calculate-inverse-modular-exponentation-in-c

2. Stack Overflow. (2013, August 7). Computing Euler's totient function. Retrieved October 16, 2023, from https://stackoverflow.com/questions/18114138/computing-eulers-totient-function

3. Wikibooks. (n.d.). Algorithm implementation/Mathematics/Extended Euclidean algorithm#Modular inverse. Retrieved October 16, 2023, from https://en.wikibooks.org/wiki/Algorithm_Implementation/Mathematics/Extended_Euclidean_algorithm#Modular_inverse

4. Wikipedia. (n.d.). Modular multiplicative inverse#Modular arithmetic. Retrieved October 16, 2023, from https://en.wikipedia.org/wiki/Modular_multiplicative_inverse#Modular_arithmetic

5. Rosetta Code. (n.d.). Modular inverse#Python. Retrieved October 16, 2023, from https://www.rosettacode.org/wiki/Modular_inverse#Python

6. Wikipedia. (n.d.). Multiplicative group of integers modulo n. Retrieved October 16, 2023, from https://en.wikipedia.org/wiki/Multiplicative_group_of_integers_modulo_n

7. MathWorld. (n.d.). Modulo multiplication group. Retrieved October 16, 2023, from https://mathworld.wolfram.com/ModuloMultiplicationGroup.html

8. Arapura, D. (n.d.). Notes on algebra. Retrieved October 16, 2023, from https://www.math.purdue.edu/~arapura/algebra/algebra8.pdf

9. Wikipedia. (n.d.). Extended Euclidean algorithm. Retrieved October 16, 2023, from https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm